In [ ]:
import pandas as pd
import numpy as np
import warnings
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import matthews_corrcoef

In [ ]:
# Save the train and test DataFrames
X_TRAIN_DIR = "processed_data/glcm/X_train.pkl"
y_TRAIN_DIR = "processed_data/glcm/y_train.pkl"
X_TEST_DIR = "processed_data/glcm/X_test.pkl"
y_TEST_DIR = "processed_data/glcm/y_test.pkl"

X_train = pd.read_pickle(X_TRAIN_DIR)
y_train = pd.read_pickle(y_TRAIN_DIR)
X_test = pd.read_pickle(X_TEST_DIR)
y_test = pd.read_pickle(y_TEST_DIR)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
def xgboost_classification(X_train, y_train, X_test, y_test, params):
    """
    Perform XGBoost multiclass classification, evaluate the model, and create a confusion matrix.

    Parameters:
    X_train (torch.Tensor): Training features.
    y_train (torch.Tensor): Training labels.
    X_test (torch.Tensor): Testing features.
    y_test (torch.Tensor): Testing labels.

    Returns:
    None
    """

    warnings.filterwarnings("ignore", category=FutureWarning, module="xgboost")
    
    # Create a label encoder
    label_encoder = LabelEncoder()
    
    # Encode the class labels
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)
    
    # Inverse transform for class labels in the confusion matrix
    class_labels = label_encoder.classes_
    
    # Create a DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train_encoded)
    dtest = xgb.DMatrix(X_test)

    num_round = 100  # Number of boosting rounds

    # Train the XGBoost model
    model = xgb.train(params, dtrain, num_round)

   # Predict labels
    y_pred_prob = model.predict(dtest)
    print("Shape of y_pred_prob:", y_pred_prob.shape)
    y_pred_class = np.argmax(y_pred_prob, axis=1)  # Find the index of the maximum probability for each sample
    y_pred = [class_labels[idx] for idx in y_pred_class]  # Convert indices to class labels

    # Create a confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Plot confusion matrix as a heatmap
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.show()

    # Calculate MCC using true labels and predicted labels
    mcc = matthews_corrcoef(y_test, y_pred)
    print("Matthews Correlation Coefficient:", mcc)

    # Print classification report
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    

In [ ]:
# Define parameters for the XGBoost model
base_params = {
    'objective': 'multi:softprob',  # Multiclass classification
    'num_class': 5,  # Number of classes
    'max_depth': 6,  # Maximum depth of trees
    'eta': 0.3,  # Learning rate
}

# Call the xgboost_classification function with your data
xgboost_classification(X_train, y_train, X_test, y_test, base_params)

In [ ]:
import optuna
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, matthews_corrcoef, classification_report
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def objective(trial, X_train, y_train, X_test, y_test):
    # Create a label encoder
    label_encoder = LabelEncoder()
    
    # Encode the class labels
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)
    
    # Inverse transform for class labels in the confusion matrix
    class_labels = label_encoder.classes_
    
    # Create a DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train_encoded)
    dtest = xgb.DMatrix(X_test)

    params = {
        'objective': 'multi:softprob',
        'num_class': 5,
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'eta': trial.suggest_float('eta', 0.01, 1.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'lambda': trial.suggest_int('lambda', 0, 100),
        'alpha': trial.suggest_int('alpha', 0, 100),
    }

    num_round = 100

    # Train the XGBoost model
    model = xgb.train(params, dtrain, num_round)

    # Predict labels
    y_pred_prob = model.predict(dtest)
    y_pred_class = np.argmax(y_pred_prob, axis=1)  # Find the index of the maximum probability for each sample
    y_pred = [class_labels[idx] for idx in y_pred_class]  # Convert indices to class labels

    # Calculate MCC using true labels and predicted labels
    mcc = matthews_corrcoef(y_test, y_pred)

    # Return the negative MCC since Optuna minimizes the objective function
    return -mcc

# Example usage in an Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, X_train, y_train, X_test, y_test), n_trials=100)

# Get the best parameters
best_params = study.best_params
print("Best Parameters:", best_params)


In [ ]:
xgboost_classification(X_train, y_train, X_test, y_test, best_params)

In [ ]:
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt

# Load your data and split it into training and testing sets
# (Assuming you have already loaded and split your data)

label_encoder = LabelEncoder()

# Encode the class labels
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Inverse transform for class labels in the confusion matrix
class_labels = label_encoder.classes_

# Train the XGBoost model
dtrain = xgb.DMatrix(X_train, label=y_train)
params = {
    'objective': 'multi:softprob',
    'num_class': len(class_labels),
    'max_depth': 6,
    'eta': 0.3,
    'subsample': 1,
    'colsample_bytree': 0.7,
    # Add other hyperparameters as needed
}
num_round = 100
model = xgb.train(params, dtrain, num_round)

# Get feature importance scores
importance = model.get_score(importance_type='weight')  # 'weight' is one of the importance types

# Convert the importance scores to a DataFrame for easier manipulation and visualization
feature_importance = pd.DataFrame.from_dict(importance, orient='index', columns=['importance'])
feature_importance = feature_importance.sort_values(by='importance', ascending=False)

# Plot the feature importance
plt.figure(figsize=(10, 6))
feature_importance.head(20).plot(kind='barh')
plt.title("Top 20 Feature Importance")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.show()


In [ ]:
# Visualize the correlation matrix as a heatmap
import seaborn as sns
import matplotlib.pyplot as plt

correlation_matrix = X_train.corr()

threshold = 0.8
highly_collinear_pairs = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > threshold:
            feature_i = correlation_matrix.columns[i]
            feature_j = correlation_matrix.columns[j]
            highly_collinear_pairs.add((correlation_matrix.iloc[i, j],feature_i, feature_j))

print(len(highly_collinear_pairs))

# Print out highly collinear feature pairs
for corr, feature_i, feature_j in highly_collinear_pairs:
    print(f"Correlation: {corr:.2f}, Feature 1: {feature_i}, Feature 2: {feature_j}")

